In [58]:
import os
import asyncio
import yaml
import json
import re
import textwrap
import base64
import openlit
from dotenv import load_dotenv
from semantic_kernel import Kernel
from semantic_kernel.agents import ChatCompletionAgent, AgentGroupChat
from semantic_kernel.agents.strategies import KernelFunctionTerminationStrategy, KernelFunctionSelectionStrategy
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion, AzureChatPromptExecutionSettings
from semantic_kernel.connectors.search_engine import GoogleConnector
from semantic_kernel.core_plugins import WebSearchEnginePlugin
from semantic_kernel.functions import KernelFunctionFromPrompt, KernelArguments
from pydantic import BaseModel, Field, ValidationError
from typing import List
from IPython.display import display, Markdown

load_dotenv()

True

In [59]:
observability = True
if observability :
    import asyncio
import logging

from opentelemetry._logs import set_logger_provider
from opentelemetry.exporter.otlp.proto.grpc._log_exporter import OTLPLogExporter
from opentelemetry.exporter.otlp.proto.grpc.metric_exporter import OTLPMetricExporter
from opentelemetry.exporter.otlp.proto.grpc.trace_exporter import OTLPSpanExporter
from opentelemetry.metrics import set_meter_provider
from opentelemetry.sdk._logs import LoggerProvider, LoggingHandler
from opentelemetry.sdk._logs.export import BatchLogRecordProcessor
from opentelemetry.sdk.metrics import MeterProvider
from opentelemetry.sdk.metrics.export import PeriodicExportingMetricReader
from opentelemetry.sdk.metrics.view import DropAggregation, View
from opentelemetry.sdk.resources import Resource
from opentelemetry.sdk.trace import TracerProvider
from opentelemetry.sdk.trace.export import BatchSpanProcessor
from opentelemetry.semconv.resource import ResourceAttributes
from opentelemetry.trace import set_tracer_provider




# Endpoint to the Aspire Dashboard
endpoint = "http://localhost:4317"

# Create a resource to represent the service/sample
resource = Resource.create({ResourceAttributes.SERVICE_NAME: "telemetry-aspire-dashboard-quickstart"})


def set_up_logging():
    exporter = OTLPLogExporter(endpoint=endpoint)

    # Create and set a global logger provider for the application.
    logger_provider = LoggerProvider(resource=resource)
    # Log processors are initialized with an exporter which is responsible
    # for sending the telemetry data to a particular backend.
    logger_provider.add_log_record_processor(BatchLogRecordProcessor(exporter))
    # Sets the global default logger provider
    set_logger_provider(logger_provider)

    # Create a logging handler to write logging records, in OTLP format, to the exporter.
    handler = LoggingHandler()
    # Add filters to the handler to only process records from semantic_kernel.
    handler.addFilter(logging.Filter("semantic_kernel"))
    # Attach the handler to the root logger. `getLogger()` with no arguments returns the root logger.
    # Events from all child loggers will be processed by this handler.
    logger = logging.getLogger()
    logger.addHandler(handler)
    logger.setLevel(logging.INFO)


def set_up_tracing():
    exporter = OTLPSpanExporter(endpoint=endpoint)

    # Initialize a trace provider for the application. This is a factory for creating tracers.
    tracer_provider = TracerProvider(resource=resource)
    # Span processors are initialized with an exporter which is responsible
    # for sending the telemetry data to a particular backend.
    tracer_provider.add_span_processor(BatchSpanProcessor(exporter))
    # Sets the global default tracer provider
    set_tracer_provider(tracer_provider)


def set_up_metrics():
    exporter = OTLPMetricExporter(endpoint=endpoint)

    # Initialize a metric provider for the application. This is a factory for creating meters.
    meter_provider = MeterProvider(
        metric_readers=[PeriodicExportingMetricReader(exporter, export_interval_millis=5000)],
        resource=resource,
        views=[
            # Dropping all instrument names except for those starting with "semantic_kernel"
            View(instrument_name="*", aggregation=DropAggregation()),
            View(instrument_name="semantic_kernel*"),
        ],
    )
    # Sets the global default meter provider
    set_meter_provider(meter_provider)


# This must be done before any other telemetry calls
set_up_logging()
set_up_tracing()
set_up_metrics()



  

Failed to instrument openai: No module named 'openai.resources.responses'
DependencyConflict: requested: "cohere >= 5.14.0" but found: "cohere 5.13.12"
/opt/conda/envs/dev/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.
/opt/conda/envs/dev/lib/python3.11/site-packages/pydantic/_internal/_config.py:345: UserWarning: Valid config keys have changed in V2:
* 'fields' has been removed
  warnings.warn(message, UserWarning)
2025-04-01 18:38:51,171 - 140088696608000 - instrumentor.py-instrumentor:10

2025-04-01 18:39:41,921 - 140086947489472 - __init__.py-__init__:219 - ERROR: Failed to export batch code: 404, reason: <!DOCTYPE html><html lang="en"><head><meta charSet="utf-8"/><meta name="viewport" content="width=device-width"/><meta name="next-head-count" content="2"/><link rel="preload" href="/_next/static/css/0b9f113e135a0e79.css" as="style"/><link rel="stylesheet" href="/_next/static/css/0b9f113e135a0e79.css" data-n-g=""/><noscript data-n-css=""></noscript><script defer="" nomodule="" src="/_next/static/chunks/polyfills-42372ed130431b0a.js"></script><script src="/_next/static/chunks/webpack-c91adb34796940e3.js" defer=""></script><script src="/_next/static/chunks/framework-c511aa5252cf0ba6.js" defer=""></script><script src="/_next/static/chunks/main-32b51fff0a416685.js" defer=""></script><script src="/_next/static/chunks/pages/_app-d5d8f035227b6c53.js" defer=""></script><script src="/_next/static/chunks/pages/_error-95ac29e5a92b4557.js" defer=""></script><script src="/_next/stat

In [60]:
class SocialMediaPost(BaseModel):
    platform: str = Field(..., description="The social media platform where the post will be published (e.g., Twitter, LinkedIn).")
    content: str = Field(..., description="The content of the social media post, including any hashtags or mentions.")

class Article(BaseModel):
    title: str
    content: str
    call_to_action: str

class ContentOutput(BaseModel):
    article: Article
    social_media_posts: List[SocialMediaPost]

In [61]:
service_id: str = "default_service_id"
def create_kernel(service_id= service_id) -> Kernel:
    kernel = Kernel()
    kernel.add_service(AzureChatCompletion(service_id=service_id))


    return kernel

kernel = create_kernel()

In [62]:
from agent_instructions import (
    MARKET_NEWS_MONITOR_AGENT_INSTRUCTIONS,
    DATA_ANALYST_AGENT_INSTRUCTIONS,
    CONTENT_CREATOR_AGENT_INSTRUCTIONS,
    QUALITY_ASSURANCE_AGENT_INSTRUCTIONS,
)

def format_instruction(instruction_template: str, subject: str) -> str:
    return instruction_template.format(subject=subject)

subject = "Inflation in the US and the impact on the stock market in 2024"

market_news_instr = format_instruction(MARKET_NEWS_MONITOR_AGENT_INSTRUCTIONS, subject)
data_analyst_instr = format_instruction(DATA_ANALYST_AGENT_INSTRUCTIONS, subject)
content_creator_instr = format_instruction(CONTENT_CREATOR_AGENT_INSTRUCTIONS, subject)
quality_assurance_instr = format_instruction(QUALITY_ASSURANCE_AGENT_INSTRUCTIONS, subject)


In [63]:
# google_scraper_plugin.py
from semantic_kernel.functions import kernel_function
from googleapiclient.discovery import build
import requests
from bs4 import BeautifulSoup
from typing import List
import os

class WebScraperPlugin:
    def __init__(self, api_key: str, cse_id: str):
        self.api_key = api_key
        self.cse_id = cse_id

    @kernel_function(
        name="google_search",
        description="Performs a Google search and returns a list of URLs related to the query."
    )
    async def google_search(self, query: str, num_results: int = 5) -> List[str]:
        service = build("customsearch", "v1", developerKey=self.api_key)
        res = service.cse().list(q=query, cx=self.cse_id, num=num_results).execute()
        urls = [item['link'] for item in res.get("items", [])]
        return urls

    @kernel_function(
        name="scrape_url",
        description="Scrapes a given URL and returns the title and the first few paragraphs of text."
    )
    async def scrape_url(self, url: str) -> dict:
        try:
            headers = {
                "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
            }
            response = requests.get(url, headers=headers, timeout=10)
            response.raise_for_status()
            soup = BeautifulSoup(response.text, "html.parser")

            title = soup.title.string.strip() if soup.title else "No Title"
            paragraphs = soup.find_all("p")
            text_content = "\n".join(p.get_text(strip=True) for p in paragraphs[:3])  # First 3 paragraphs

            return {
                "url": url,
                "title": title,
                "text": text_content
            }
        except Exception as e:
            return {
                "url": url,
                "error": str(e)
            }

web_scraper_tool = WebScraperPlugin(
    api_key=os.getenv("GOOGLE_API_KEY"),
    cse_id=os.getenv("GOOGLE_SEARCH_ENGINE_ID")
)


In [64]:
selection_function = KernelFunctionFromPrompt(
    function_name="selection",
    prompt="""
    Based on the conversation history, determine the next agent to take a turn. 
    Only return the name of the agent from the following list:
    - MarketNewsMonitor
    - DataAnalyst
    - ContentCreator
    - ContentOfficer

    Rules:
    - MarketNewsMonitor starts first.
    - After MarketNewsMonitor, DataAnalyst speaks.
    - After DataAnalyst, ContentCreator speaks.
    - After ContentCreator, ContentOfficer speaks.
    - If ContentOfficer requests changes, restart the cycle with MarketNewsMonitor.

    History:
    {{$history}}
    """,
)

In [65]:
termination_function = KernelFunctionFromPrompt(
    function_name="termination",
    prompt="""
    Determine if the structured content output (article and social_media_posts) is complete and valid JSON. 
    If the output is complete and valid, respond with "yes". Otherwise, respond with "no".

    History:
    {{$history}}
    """
)

In [66]:
output_settings = AzureChatPromptExecutionSettings(response_format=ContentOutput)


In [67]:
market_news_agent = ChatCompletionAgent(kernel=kernel,
                                         name="MarketNewsMonitor",
                                           instructions=market_news_instr,
                                            plugins=[web_scraper_tool]
                                           )
data_analyst_agent = ChatCompletionAgent(kernel=kernel, 
                                         name="DataAnalyst",
                                          instructions=data_analyst_instr,
                                          plugins=[web_scraper_tool])
content_creator_agent = ChatCompletionAgent(kernel=kernel, name="ContentCreator", instructions=content_creator_instr)
quality_assurance_agent = ChatCompletionAgent(kernel=kernel,
                                                 name="ContentOfficer",
                                                 instructions=quality_assurance_instr,
                                                 arguments=KernelArguments(settings=output_settings))

agent_group = AgentGroupChat(
    agents=[market_news_agent, data_analyst_agent, content_creator_agent, quality_assurance_agent],
    termination_strategy = KernelFunctionTerminationStrategy(
        agents=[quality_assurance_agent],
        function=termination_function,
        kernel=kernel,
        result_parser=lambda result: any("yes" in item.content.lower() for item in result.value) if isinstance(result.value, list) else "yes" in result.value.content.lower(),
        history_variable_name="history",
        maximum_iterations=10
    ),
    selection_strategy=KernelFunctionSelectionStrategy(
        function=selection_function,
        kernel=kernel,
        result_parser=lambda result: result.value[0].content.strip() if isinstance(result.value, list) and result.value else result.value.content.strip(),
        agent_variable_name="agents",
        history_variable_name="history"
    )
)

In [68]:
async def main(subject: str):

    initial_message = f"Subject: {subject}\n\nCollaborate to produce structured content output matching the provided schema."
    await agent_group.add_chat_message(message=initial_message)

    final_response = ""
    async for content in agent_group.invoke():
        print(f"# {content.name}: {content.content}\n")
        final_response = content.content

    return final_response

In [69]:
final_output = await main("Inflation in the US and the impact on the stock market in 2024")

2025-04-01 18:38:52,113 - 140088696608000 - __init__.py-__init__:100 - WARNING: Invalid type NoneType for attribute 'gen_ai.response.service_tier' value. Expected one of ['bool', 'str', 'bytes', 'int', 'float'] or a sequence of those types
2025-04-01 18:38:54,432 - 140088696608000 - __init__.py-__init__:100 - WARNING: Invalid type NoneType for attribute 'gen_ai.response.service_tier' value. Expected one of ['bool', 'str', 'bytes', 'int', 'float'] or a sequence of those types
2025-04-01 18:38:59,404 - 140088696608000 - async_openai.py-async_openai:875 - ERROR: Error in trace creation: 'content'
2025-04-01 18:39:19,062 - 140088696608000 - async_openai.py-async_openai:875 - ERROR: Error in trace creation: 'content'


# MarketNewsMonitor: ### Detailed Summary Report: Inflation in the US and the Impact on the Stock Market in 2024

#### Overview
The ongoing dynamics of US inflation and their anticipated impact on the stock market in 2024 present a complex financial landscape influenced by rising interest rates and the potential for new tariff implementations. Below, we distill key insights from recent analyses, revealing critical considerations investors must weigh as they navigate these economic conditions.

#### Key Insights

1. **Interest Rates and Stock Market Dynamics**
   - **Source**: [U.S. Bank Article](https://www.usbank.com/investing/financial-perspectives/market-news/how-do-rising-interest-rates-affect-the-stock-market.html)
   - **Date**: February 20, 2025
   - **Summary**: Despite stabilizing interest rates, they remain at elevated levels, impacting stock valuations and investment strategies. Elevated rates typically increase borrowing costs for companies, which can dampen corporate earni

2025-04-01 18:39:26,999 - 140088696608000 - __init__.py-__init__:100 - WARNING: Invalid type NoneType for attribute 'gen_ai.response.service_tier' value. Expected one of ['bool', 'str', 'bytes', 'int', 'float'] or a sequence of those types
2025-04-01 18:39:38,458 - 140088696608000 - async_openai.py-async_openai:875 - ERROR: Error in trace creation: 'content'


# DataAnalyst: To explore the data visualizations and resources that accompany this report, please refer to the following articles:

- **U.S. Bank Article on Interest Rates**: [Read Here](https://www.usbank.com/investing/financial-perspectives/market-news/how-do-rising-interest-rates-affect-the-stock-market.html).
- **Goldman Sachs Article on Tariffs**: [Read Here](https://www.goldmansachs.com/insights/articles/how-tariffs-are-forecast-to-affect-us-stocks).

These insights will help guide your understanding and approach the content creation strategy effectively in relation to U.S. inflation and stock market impacts in 2024.



2025-04-01 18:39:39,672 - 140088696608000 - __init__.py-__init__:100 - WARNING: Invalid type NoneType for attribute 'gen_ai.response.service_tier' value. Expected one of ['bool', 'str', 'bytes', 'int', 'float'] or a sequence of those types
2025-04-01 18:39:55,602 - 140088696608000 - async_openai.py-async_openai:875 - ERROR: Error in trace creation: 'content'


# ContentCreator: ### Blog Post Draft: Navigating the Inflation Wave in 2024

#### Title: Understanding the Looming Impact of U.S. Inflation on the Stock Market in 2024

As we sail into 2024, investors are turning their eyes towards key economic signifiers to navigate the uncertain waters of the stock market. Two critical phenomena stand out: persistent inflation and the evolving landscape of international trade tariffs.

**Inflation Dynamics**

Interest rates have been a focal point in the current economic dialogue. Insights from a recent [U.S. Bank article](https://www.usbank.com/investing/financial-perspectives/market-news/how-do-rising-interest-rates-affect-the-stock-market.html) indicate that, although interest rates have stabilized, they remain significantly elevated. This elevation impacts borrowing costs across industries, crucially affecting companies' earnings. Investors should remain vigilant, as these elevated rates can swiftly alter stock market dynamics, particularly in s

2025-04-01 18:39:56,648 - 140088696608000 - __init__.py-__init__:100 - WARNING: Invalid type NoneType for attribute 'gen_ai.response.service_tier' value. Expected one of ['bool', 'str', 'bytes', 'int', 'float'] or a sequence of those types
2025-04-01 18:40:09,534 - 140088696608000 - async_openai.py-async_openai:875 - ERROR: Error in trace creation: 'content'
2025-04-01 18:40:10,621 - 140088696608000 - __init__.py-__init__:100 - WARNING: Invalid type NoneType for attribute 'gen_ai.response.service_tier' value. Expected one of ['bool', 'str', 'bytes', 'int', 'float'] or a sequence of those types


# ContentOfficer: {"article":{"title":"Understanding the Looming Impact of U.S. Inflation on the Stock Market in 2024","content":"As we sail into 2024, investors are turning their eyes towards key economic signifiers to navigate the uncertain waters of the stock market. Two critical phenomena stand out: persistent inflation and the evolving landscape of international trade tariffs.\n\n#### Inflation Dynamics\n\nInterest rates have been a focal point in the current economic dialogue. Insights from a recent [U.S. Bank article](https://www.usbank.com/investing/financial-perspectives/market-news/how-do-rising-interest-rates-affect-the-stock-market.html) indicate that, although interest rates have stabilized, they remain significantly elevated. This elevation impacts borrowing costs across industries, crucially affecting companies' earnings. Investors should remain vigilant, as these elevated rates can swiftly alter stock market dynamics, particularly in sectors traditionally resilient to s

In [70]:
final_output

'{"article":{"title":"Understanding the Looming Impact of U.S. Inflation on the Stock Market in 2024","content":"As we sail into 2024, investors are turning their eyes towards key economic signifiers to navigate the uncertain waters of the stock market. Two critical phenomena stand out: persistent inflation and the evolving landscape of international trade tariffs.\\n\\n#### Inflation Dynamics\\n\\nInterest rates have been a focal point in the current economic dialogue. Insights from a recent [U.S. Bank article](https://www.usbank.com/investing/financial-perspectives/market-news/how-do-rising-interest-rates-affect-the-stock-market.html) indicate that, although interest rates have stabilized, they remain significantly elevated. This elevation impacts borrowing costs across industries, crucially affecting companies\' earnings. Investors should remain vigilant, as these elevated rates can swiftly alter stock market dynamics, particularly in sectors traditionally resilient to such financia

In [71]:
def extract_json(text: str) -> dict:
    pattern = re.compile(r"```json\s*(\{.*\})\s*```", re.DOTALL)
    match = pattern.search(text)
    if match:
        return json.loads(match.group(1))
    return json.loads(text.strip())

output_data = extract_json(final_output)
article = output_data.get("article", {})
posts = output_data.get("social_media_posts", [])

article_md = f"# {article.get('title')}\n\n{article.get('content')}\n\n**{article.get('call_to_action')}**"
display(Markdown(article_md))

for post in posts:
    print(f"Platform: {post['platform']}")
    print(textwrap.fill(post['content'], width=60))
    print('-' * 60)

# Understanding the Looming Impact of U.S. Inflation on the Stock Market in 2024

As we sail into 2024, investors are turning their eyes towards key economic signifiers to navigate the uncertain waters of the stock market. Two critical phenomena stand out: persistent inflation and the evolving landscape of international trade tariffs.

#### Inflation Dynamics

Interest rates have been a focal point in the current economic dialogue. Insights from a recent [U.S. Bank article](https://www.usbank.com/investing/financial-perspectives/market-news/how-do-rising-interest-rates-affect-the-stock-market.html) indicate that, although interest rates have stabilized, they remain significantly elevated. This elevation impacts borrowing costs across industries, crucially affecting companies' earnings. Investors should remain vigilant, as these elevated rates can swiftly alter stock market dynamics, particularly in sectors traditionally resilient to such financial climates.

#### Tariffs and International Trade

Meanwhile, tariffs continue to play a pivotal role in shaping the stock market landscape. According to a [Goldman Sachs analysis](https://www.goldmansachs.com/insights/articles/how-tariffs-are-forecast-to-affect-us-stocks), proposed tariffs have the power to chip away at the earnings of major U.S. companies significantly. The possibility of new tariffs affecting North American and European trade could severely alter corporate strategies, requiring a recalibration of global market priorities and increasing the effective tariff rate substantially.

#### Mitigating Strategies for Investors

For investors, strategic adjustments are imperative:
- **Interest Rate Resilience**: Focus on sectors such as utilities and healthcare, which traditionally withstand rate hikes.
- **Diversified Portfolios**: Consider diversifying into domestic-oriented companies less vulnerable to international tariff policies.
- **Hedging Trade Risks**: Implement hedging structures to mitigate potential adverse impacts from ongoing trade negotiations.

#### Conclusion

As we look ahead, staying informed and adapting investment strategies will be vital for navigating the challenges inflation and trade tariffs pose. Proactive measures and an informed understanding of these macroeconomic forces can empower investors to make strategic decisions that bolster confidence and foster growth in their investment portfolios in 2024.

**Explore more insights and stay updated with our latest articles to adapt your investment strategies effectively.**

Platform: Twitter
🔍 Discover how US inflation dynamics and interest rate
changes are setting the stage for 2024's stock market
movements. Insightful analysis reveals pathways for
strategic investment. [Link to Blog Post]
------------------------------------------------------------
Platform: LinkedIn
In 2024, US inflation and tariff negotiations are key
factors shaping stock market trends. Explore our in-depth
analysis of how these forces can impact your portfolio and
strategic opportunities to leverage these changes for
growth. [Link to Blog Post]
------------------------------------------------------------
Platform: Instagram
Swipe up to dive deep into how inflation and tariffs are
influencing the US stock market in 2024. Stay ahead with
strategic insights and recommendations. [Link to Blog Post]
------------------------------------------------------------
Platform: Facebook
As we step into 2024, understanding the intersecting
influences of inflation and international tariffs on the U